<a href="https://colab.research.google.com/github/arkeodev/nlp/blob/main/simple_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple Transformers Architecture